In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patheffects as pe
from matplotlib.ticker import FormatStrFormatter

import os; print(os.getcwd())

import result_utils
import corner

/net/lem/data1/regt/retrieval_base/Luhman_16/figures


In [2]:
name_A = 'Luhman 16A'; name_B = 'Luhman 16B'
color_A = '#FF622E'; color_B = '#396ED8'

lw_d = 0.8
lw_m = 1.0

res_alpha = 0.85

cmap_A = mpl.colors.LinearSegmentedColormap.from_list('', ['w',color_A])
env_colors_A = cmap_A([0,1/3,2/3,1])
env_colors_A[:,3] = 0.5
env_colors_A[0,3] = 0.0

cmap_B = mpl.colors.LinearSegmentedColormap.from_list('', ['w',color_B])
env_colors_B = cmap_B([0,1/3,2/3,1])
env_colors_B[:,3] = 0.5
env_colors_B[0,3] = 0.0

A = result_utils.RetrievalResults('/net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_A_ret_22_1column_n1000/test_')
B = result_utils.RetrievalResults('/net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_B_ret_62_1column_n1000/test_')

/net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_A_ret_22_1column_n1000/test_data
  analysing data from /net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_A_ret_22_1column_n1000/test_.txt
/net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_B_ret_62_1column_n1000/test_data
  analysing data from /net/schenk/data2/regt/data_backup/Luhman_16/retrieval_outputs/J_B_ret_62_1column_n1000/test_.txt


In [3]:
def get_log_g(Res):
    from retrieval_base.utils import sc
    
    idx_M_p = Res.ParamTable.get('M_p', key='idx_free')
    idx_R_p = Res.ParamTable.get('R_p', key='idx_free')
    
    M_p = np.copy(Res.posterior[:,idx_M_p])
    R_p = np.copy(Res.posterior[:,idx_R_p])

    # Compute the surface gravity
    M_p *= (sc.m_jup*1e3)
    R_p *= (sc.r_jup_mean*1e2)

    g = (sc.G*1e3) * M_p/R_p**2
    log_g = np.log10(g)

    return log_g

get_log_g(A)

array([5.09128391, 5.12326525, 5.09949387, ..., 5.09290277, 5.0905325 ,
       5.09492422])

In [4]:
def get_ranges(posterior, q=[0.16,0.84], n=4):
    
    posterior_range = np.quantile(posterior, q=q, axis=0)
    med = np.median(posterior, axis=0)
    
    posterior_range -= med
    posterior_range *= n
    posterior_range += med

    return posterior_range.T

to_plot = {
    'rv': r'$v_\mathrm{rad}$', 
    'vsini': r'$v\ \sin{i}$',
    #'M_p': r'$M_\mathrm{p}$',
    #'R_p': r'$R_\mathrm{p}$',
    'log_g': r'$\log\ g$',
    'log_H2O': r'$\log\ \mathrm{H_2O}$', 
    'log_K': r'$\log\ \mathrm{K}$', 
    'log_Na': r'$\log\ \mathrm{Na}$', 
    'log_HF': r'$\log\ \mathrm{HF}$', 
    'log_FeH': r'$\log\ (\mathrm{FeH})_0$', 
}

indices = [A.ParamTable.get(name, key='idx_free') for name in to_plot.keys()]
post_A = A.posterior[:,indices]
#post_A = np.insert(post_A, 2, get_log_g(A), axis=1)
post_A = post_A[:,::-1]
ranges_A = get_ranges(post_A)

indices = [B.ParamTable.get(name, key='idx_free') for name in to_plot.keys()]
post_B = B.posterior[:,indices]
#post_B = np.insert(post_B, 2, get_log_g(B), axis=1)
ranges_B = get_ranges(post_B)

kwargs = dict(
    bins=20, fill_contours=True, plot_datapoints=True, labels=None,  max_n_ticks=3, 
    label_kwargs={'rotation':0.}, 
)

# Make offset sub-figures
nrows, ncols = 22, 19
idx = 18

fig = plt.figure(figsize=(9,9*nrows/ncols))
gs = fig.add_gridspec(nrows=nrows, ncols=ncols, hspace=0., wspace=0)
subfig = np.array([fig.add_subfigure(gs[-idx:,:idx]), fig.add_subfigure(gs[:idx,-idx:]),])
for subfig_i in subfig:
    subfig_i.set_facecolor('none')

# Corner-plot in each subfigure
kwargs['color']          = color_A
kwargs['hist_kwargs']    = {'edgecolor':color_A, 'facecolor':env_colors_A[1], 'fill':True}
kwargs['contour_kwargs'] = {'linewidths':1.0, 'color':color_A}
kwargs['reverse'] = True
subfig[1] = corner.corner(fig=subfig[1], data=post_A, range=ranges_A, **kwargs)

# Corner-plot in each subfigure
kwargs['color']          = color_B
kwargs['hist_kwargs']    = {'edgecolor':color_B, 'facecolor':env_colors_B[1], 'fill':True}
kwargs['contour_kwargs'] = {'linewidths':1.0, 'color':color_B}
kwargs['reverse'] = False
subfig[0] = corner.corner(fig=subfig[0], data=post_B, range=ranges_B, **kwargs)


# x/y-label kwargs and padding
ann_kwargs = dict(xycoords='axes fraction', ha='center', va='center')
labelpad = 0.6

ann_name_kwargs = dict(
    xycoords='axes fraction', clip_on=False, fontsize=15, fontweight='bold',
    #path_effects=[pe.withStroke(linewidth=5, foreground='w', alpha=0.9)], 
)

# Luhman 16B subfigure ----------------------------------------------------------
ax = np.array(subfig[0].axes)
ax = ax.reshape((int(np.sqrt(len(ax))), int(np.sqrt(len(ax)))))

subfig[0].subplots_adjust(wspace=0.0, hspace=0.0) # Remove spacing between axes
[ax_ij.tick_params(top=True, right=True, bottom=True, left=True, direction='inout') for ax_ij in ax.ravel()]

#labels = np.insert([to_plot[key] for key in to_plot.keys()], 2, r'$\log\ g$')
labels = [to_plot[key] for key in to_plot.keys()]
for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        
        xlim = ranges_B[j]
        ylim = ranges_B[i]

        if i != 0 and j == 0:
            # Add y-labels to left column except for diagonal
            ax[i,j].annotate(labels[i], xy=(-labelpad,0.5), rotation=90, **ann_kwargs)
        if i == len(ax)-1:
            # Add x-labels to bottom row
            ax[i,j].annotate(labels[j], xy=(0.5,-labelpad), **ann_kwargs)
        if i == j:
            # Title above diagonal axes
            ax[i,j].annotate(labels[i], xy=(0.5,1+0.15), **ann_kwargs)
            ylim = None
        
        ax[i,j].set(xlim=xlim, ylim=ylim)
        [l.set_rotation_mode('anchor') for l in ax[i,j].get_yticklabels()]

ax[0,0].annotate(
    'Luhman 16B', xy=(0.15,1), xytext=(-0.7,1.25), ha='left', va='bottom', color=color_B, 
    arrowprops={
        'arrowstyle':'-', 'connectionstyle':'angle3,angleA=90,angleB=-30', 
        'shrinkA':0, 'shrinkB':0, 'lw':1.5, 'color':color_B, 
        }, **ann_name_kwargs
    )

# Luhman 16A subfigure ----------------------------------------------------------
ax = np.array(subfig[1].axes)
ax = ax.reshape((int(np.sqrt(len(ax))), int(np.sqrt(len(ax)))))

subfig[1].subplots_adjust(wspace=0.0, hspace=0.0) # Remove spacing between axes
[ax_ij.tick_params(top=True, right=True, bottom=True, left=True, direction='inout') for ax_ij in ax.ravel()]

#labels = np.insert([to_plot[key] for key in to_plot.keys()], 2, r'$\log\ g$')[::-1]
for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        
        xlim = ranges_A[::-1][j]
        ylim = ranges_A[::-1][i]

        if i == 0:
            # Add x-labels to top row
            ax[i,j].annotate(labels[j], xy=(0.5,1+labelpad), **ann_kwargs)
        if i != len(ax)-1 and j == len(ax)-1:
            # Add y-labels to right column except for diagonal
            ax[i,j].annotate(labels[i], xy=(1+labelpad,0.5), rotation=90, **ann_kwargs)
        if i == j:
            # Title below diagonal axes
            ax[i,j].annotate(labels[i], xy=(0.5,-0.19), **ann_kwargs)
            ylim = None

        ax[i,j].set(xlim=xlim, ylim=ylim)
        [l.set_rotation_mode('anchor') for l in ax[i,j].get_yticklabels()]

ax[0,0].annotate(
    'Luhman 16A', xy=(0.15,0), xytext=(-0.7,-0.3), ha='left', va='top', color=color_A, 
    arrowprops={
        'arrowstyle':'-', 'connectionstyle':'angle3,angleA=90,angleB=30', 
        'shrinkA':0, 'shrinkB':0, 'lw':1.5, 'color':color_A, 
        }, **ann_name_kwargs
    )

# Adjust the margins
margin = 0.08
plt.subplots_adjust(left=margin, bottom=margin, top=1-margin, right=1-margin)

plt.savefig('./plots/J_band_corner_new.pdf')
#plt.show()
plt.close()

/net/lem/data2/regt/conda_pRT3_env/lib/python3.12/site-packages/corner/core.py:800: UserWarning: The following kwargs were not used by contour: 'color'
  ax.contour(X2, Y2, H2.T, V, **contour_kwargs)
/net/lem/data2/regt/conda_pRT3_env/lib/python3.12/site-packages/corner/core.py:800: UserWarning: The following kwargs were not used by contour: 'color'
  ax.contour(X2, Y2, H2.T, V, **contour_kwargs)
/net/lem/data2/regt/conda_pRT3_env/lib/python3.12/site-packages/corner/core.py:800: UserWarning: The following kwargs were not used by contour: 'color'
  ax.contour(X2, Y2, H2.T, V, **contour_kwargs)
/net/lem/data2/regt/conda_pRT3_env/lib/python3.12/site-packages/corner/core.py:800: UserWarning: The following kwargs were not used by contour: 'color'
  ax.contour(X2, Y2, H2.T, V, **contour_kwargs)
/net/lem/data2/regt/conda_pRT3_env/lib/python3.12/site-packages/corner/core.py:800: UserWarning: The following kwargs were not used by contour: 'color'
  ax.contour(X2, Y2, H2.T, V, **contour_kwargs)
